In [ ]:
%matplotlib inline
import warnings
warnings.filterwarnings("ignore")
 
import pandas as pd
import matplotlib.pyplot as plt
import re
import time
import warnings
import numpy as np
import seaborn as sns
 from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
import math
from sklearn.ensemble import RandomForestClassifier
from sklearn import model_selection

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving AirPi Data - AirPi.csv to AirPi Data - AirPi (1).csv


In [ ]:
#Reading data by using pandas
import io
data = pd.read_csv(io.StringIO(uploaded['AirPi Data - AirPi.csv'].decode('utf-8')))


In [ ]:
#Cleaning the data
regex = re.compile(r"\[|\]|<", re.IGNORECASE)
data.columns = [regex.sub("_", col) if any(x in str(col) for x in set(('[',']','<'))) else col for col in data.columns.values]
data.head()


,Date time,Volume _mV_,Light_Level _Ohms_,Temperature-DHT _Celsius_,Pressure _Hectopascal_,Temperature-BMP _Celsius_,Relative_Humidity _%_,Air_Quality _Ohms_,Carbon_Monoxide _Ohms_,Nitrogen_Dioxide _Ohms_
0,2016-11-27 17:22:45,133.064516,5064.140045,24.050000,1024.434757,24.130,37.780,40776.51523,283808.1869,14849.59631
1,2016-11-27 17:27:45,96.935484,5075.234701,24.525000,1024.494539,24.520,36.880,40661.05341,286562.2566,15359.88699
2,2016-11-27 17:32:46,97.903226,5074.103939,25.045000,1024.583963,24.895,35.705,40666.63465,293985.7963,15772.13696
3,2016-11-27 17:37:46,105.645161,5086.325929,25.334999,1024.633698,25.080,34.810,40574.72690,298587.8165,16150.72702
4,2016-11-27 17:42:46,130.806452,5086.339059,26.750000,1024.743719,25.250,34.435,40546.07795,299600.9932,16506.56816


In [ ]:
# Linear regression doesn't work on date data. Therefore we need to convert it into numerical value.
import datetime as dt
Date = data['Date time']
data['Date time'] = pd.to_datetime(data['Date time'])
data['Date time']=data['Date time'].apply(lambda x: time.mktime(x.timetuple()))


# Train,Test Splitting

In [ ]:
X = data.drop(['Air_Quality _Ohms_'], axis=1) # here droping value column wise i,e why axis=1
# Xt = X.drop(['Date time'], axis=1)

y = data['Air_Quality _Ohms_'].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30)

print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)



(10200, 9)
(4372, 9)
(10200,)
(4372,)


By looking at Air_Quality column we can find that it is a regression problem. So we can train regression models by using this dataset

# Linear Regression

In [ ]:
from sklearn.linear_model import LinearRegression
lr_reg=LinearRegression().fit(X_train, y_train)


y_pred_test_lr = lr_reg.predict(X_test)
lr_test_predictions = [round(value,5) for value in y_pred_test_lr]
print("Predictions of test data",lr_test_predictions)
y_pred_train_lr = lr_reg.predict(X_train)
lr_train_predictions = [round(value,5) for value in y_pred_train_lr]
print("Predictions of train data",lr_train_predictions)

Predictions of test data [37520.52014, 39763.98805, 37930.74657, 43584.37798, 40524.59669, 40402.8106, 41141.11791, 39815.73559, 41532.70892, 36675.78436, 39768.12719, 39797.54809, 39147.74779, 40205.34704, 40761.24175, 39816.24261, 34934.89457, 39393.46319, 39363.20358, 39152.01002, 40580.37043, 38292.5907, 41783.40077, 40140.86248, 45119.16812, 37373.61096, 37623.74753, 41318.15671, 41499.56287, 41254.30486, 37108.27272, 39294.63593, 36375.67133, 40004.60841, 38370.7526, 39897.04048, 46124.41025, 39133.05182, 38568.02342, 37628.03905, 38195.94217, 42530.07265, 37126.39319, 33751.45815, 40191.27514, 38448.58185, 38602.39913, 42575.43754, 38031.91713, 39103.45253, 39477.26344, 41132.07016, 38804.14529, 39871.27445, 39420.59567, 42975.34854, 40830.50338, 42200.57298, 36726.16157, 39934.6972, 32850.6338, 45094.5086, 36100.58454, 40499.09662, 36422.94504, 44962.20793, 41637.99665, 35038.52794, 38524.61834, 38030.09857, 37604.49487, 43832.83026, 40625.45757, 42396.39662, 38166.19577, 43271

# Random Forest Regressor

In [ ]:
from sklearn.ensemble import RandomForestRegressor
regr1 = RandomForestRegressor(max_features='sqrt',min_samples_leaf=4,min_samples_split=3,n_estimators=40, n_jobs=-1)
regr1.fit(X_train, y_train)
y_pred_test_rf = regr1.predict(X_test)
rndf_test_predictions = [round(value,5) for value in y_pred_test_rf]
print("Predictions of test data",rndf_test_predictions)
y_pred_train_rf = regr1.predict(X_train)
rndf_train_predictions = [round(value,5) for value in y_pred_train_rf]
print("Predictions of train data",rndf_train_predictions)


Predictions of test data [35329.40764, 42112.36986, 40197.98287, 42411.29999, 41387.14154, 40317.71485, 40042.20984, 41920.4631, 42391.4639, 32071.07274, 37942.18543, 40699.98476, 40705.46006, 39649.01563, 41687.00598, 36091.71322, 34404.11689, 38277.23397, 40145.92554, 39383.54974, 40402.60066, 39752.32019, 42807.94143, 39628.82536, 42263.75561, 38421.60209, 36657.92267, 36962.80768, 41527.0859, 41977.02337, 35814.52849, 39702.87399, 41906.93251, 39031.29664, 39483.19741, 41090.8801, 43756.19278, 36335.96135, 39154.51606, 34113.805, 37507.62137, 42642.59448, 38391.81416, 33172.67802, 43360.48749, 39604.35256, 37678.90506, 44630.88334, 40364.54246, 40511.04625, 38531.3284, 43360.05293, 41169.06818, 41075.17722, 40258.14885, 40937.34578, 43038.97668, 42004.38909, 35945.05418, 38938.88442, 35576.41903, 43840.95614, 34121.44985, 40172.27851, 34684.02817, 44150.00436, 42681.15722, 31951.22169, 35312.69037, 40780.60376, 37214.02449, 42647.70222, 40141.69541, 43184.04353, 38259.95043, 42898.

Hyper parameter tuning using RandomsearchCV

In [ ]:
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 50, stop = 100, num = 10)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 50, num = 10)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
# First create the base model to tune
rf = RandomForestRegressor()
# Random search of parameters, using 3 fold cross validation, 
# search across 50 different combinations, and use all available cores
rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, cv = 3, verbose=2, random_state=42, n_jobs = -1)# Fit the random search model
rf_random.fit(X_train, y_train)

rf_random.best_params_

Fitting 3 folds for each of 10 candidates, totalling 30 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  30 out of  30 | elapsed:  1.3min finished


{'bootstrap': False,
 'max_depth': 36,
 'max_features': 'sqrt',
 'min_samples_leaf': 1,
 'min_samples_split': 10,
 'n_estimators': 72}

In [ ]:
#Training model with best parameters which we got from RandomsearchCV
from sklearn.ensemble import RandomForestRegressor
regr2 = RandomForestRegressor(bootstrap = False, max_depth = 36, max_features = 'sqrt', min_samples_leaf = 1,min_samples_split=10,n_estimators=72, n_jobs=-1)
# regr1 = RandomForestRegressor(max_features='sqrt',min_samples_leaf=4,min_samples_split=3,n_estimators=40, n_jobs=-1)
regr2.fit(X_train, y_train)
y_pred_test_rf = regr2.predict(X_test)
rndf_test_predictions_ht = [round(value,5) for value in y_pred_test_rf]
print("Predictions of test data",rndf_test_predictions_ht)
y_pred_train_rf = regr2.predict(X_train)
rndf_train_predictions_ht = [round(value,5) for value in y_pred_train_rf]
print("Predictions of train data",rndf_train_predictions_ht)

Predictions of test data [35342.46249, 42108.07581, 40144.33377, 42310.03249, 41463.51041, 40616.81137, 39843.76045, 41963.17039, 42497.57374, 31581.28706, 37964.76878, 41045.94609, 40725.90474, 40148.88237, 41938.58024, 36116.56487, 34222.40304, 38238.8982, 40101.44319, 39360.2048, 40464.26625, 40020.31232, 42865.32729, 39384.53681, 42251.34884, 38978.14087, 36621.32312, 36599.5468, 41677.66004, 42125.03615, 34728.73259, 39789.16287, 41912.83676, 39319.45475, 39552.2937, 41174.43991, 43778.53782, 36363.82646, 39232.43888, 32874.53649, 37502.09133, 42472.43404, 38889.83892, 33156.00675, 43553.90993, 39823.52036, 37805.42268, 44727.40543, 40392.22761, 40538.70203, 38499.70415, 43370.98547, 41064.35107, 41209.05405, 40336.50294, 40876.85064, 43119.1194, 42023.19384, 35941.72342, 38777.52388, 35654.62173, 43747.95542, 34524.05624, 40189.76457, 34687.22624, 44315.85277, 42647.88925, 31833.78738, 35187.14644, 41145.87995, 37097.78078, 42767.95162, 40002.61646, 43340.87081, 37733.82138, 4325

# XgBoost Regressor

In [ ]:
from xgboost.sklearn import XGBRegressor
x_model = XGBRegressor(learning_rate =0.1,n_estimators=1000,max_depth=3,min_child_weight=3,gamma=0,subsample=0.8,reg_alpha=200, reg_lambda=200,colsample_bytree=0.8,nthread=4)
x_model.fit(X_train, y_train)
y_pred_test_xg = x_model.predict(X_test)
xgb_test_predictions = [round(value,5) for value in y_pred_test_xg]
print("Predictions of test data",xgb_test_predictions)
y_pred_train_xg = x_model.predict(X_train)
xgb_train_predictions = [round(value,5) for value in y_pred_train_xg]
print("Predictions of train data",xgb_train_predictions)

[04:18:31] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
Predictions of test data [35779.164, 41406.246, 40328.57, 42519.46, 41318.89, 41697.535, 40440.684, 41580.684, 42261.137, 32025.494, 38424.508, 40670.746, 39679.17, 39079.215, 41163.715, 35536.867, 34253.88, 38442.47, 40589.996, 39630.21, 41074.008, 39718.234, 42809.38, 40255.492, 42038.38, 38908.55, 37157.203, 38489.805, 41240.86, 42197.777, 36154.91, 39408.44, 41895.645, 39854.305, 39667.3, 40624.258, 43993.71, 35453.65, 38056.67, 33667.426, 37755.066, 42350.508, 37589.2, 32554.54, 43552.242, 39050.492, 37662.24, 44856.184, 40280.582, 40523.906, 39041.64, 42957.062, 41148.406, 40892.625, 40510.473, 40636.223, 43634.46, 41939.49, 35986.863, 39192.656, 34811.69, 43894.953, 33766.73, 39837.812, 34838.008, 44429.797, 42629.363, 31127.744, 35172.527, 40452.586, 37270.84, 42936.91, 40452.375, 42776.49, 37622.496, 44345.51, 34366.703, 40033.266, 40267.113, 41777.656

# Performance Metrices

### Mean Squared Error(MSE)

In [ ]:
from sklearn.metrics import mean_squared_error
from math import sqrt 
Train_RMSE = []
Test_RMSE = []

Train_RMSE.append(sqrt(mean_squared_error(y_train,lr_train_predictions)))
Train_RMSE.append(sqrt(mean_squared_error(y_train,rndf_train_predictions)))
Train_RMSE.append(sqrt(mean_squared_error(y_train,rndf_train_predictions_ht)))
Train_RMSE.append(sqrt(mean_squared_error(y_train,xgb_train_predictions)))

Test_RMSE.append(sqrt(mean_squared_error(y_test,lr_test_predictions)))
Test_RMSE.append(sqrt(mean_squared_error(y_test,rndf_test_predictions)))
Test_RMSE.append(sqrt(mean_squared_error(y_test,rndf_test_predictions_ht)))
Test_RMSE.append(sqrt(mean_squared_error(y_test,xgb_test_predictions)))

print ("  Model                                 RMSE_Train          RMSE_Test")
print ("Linear Regression -         Train: ",Train_RMSE[0],"      Test: ",Test_RMSE[0])
print ("Random Forest Regression -  Train: ",Train_RMSE[1],"      Test: ",Test_RMSE[1])
print ("RF with Hyper parm tuning - Train: ",Train_RMSE[2],"      Test: ",Test_RMSE[2])
print ("XgBoost Regression -        Train: ",Train_RMSE[3],"      Test: ",Test_RMSE[3])

  Model                                 RMSE_Train          RMSE_Test
Linear Regression -         Train:  2671.594332576108       Test:  12171.912904546683
Random Forest Regression -  Train:  5675.943176893256       Test:  5583.65128069805
RF with Hyper parm tuning - Train:  5759.711014670688       Test:  5640.216559814655
XgBoost Regression -        Train:  5678.261069575726       Test:  5606.8309119632195


 Mean Absolute Percentage Error

In [ ]:
from sklearn.metrics import mean_absolute_error
Train_MAPE=[]
Test_MAPE=[]

Train_MAPE.append((mean_absolute_error(y_train,lr_train_predictions))/(sum(y_train)/len(y_train)))
Train_MAPE.append((mean_absolute_error(y_train, rndf_train_predictions))/(sum(y_train)/len(y_train)))
Train_MAPE.append((mean_absolute_error(y_train, rndf_train_predictions_ht))/(sum(y_train)/len(y_train)))
Train_MAPE.append((mean_absolute_error(y_train, xgb_train_predictions))/(sum(y_train)/len(y_train)))

Test_MAPE.append((mean_absolute_error(y_test, lr_test_predictions))/(sum(y_test)/len(y_test)))
Test_MAPE.append((mean_absolute_error(y_test, rndf_test_predictions))/(sum(y_test)/len(y_test)))
Test_MAPE.append((mean_absolute_error(y_test, rndf_test_predictions_ht))/(sum(y_test)/len(y_test)))
Test_MAPE.append((mean_absolute_error(y_test, xgb_test_predictions))/(sum(y_test)/len(y_test)))

print ("                                         MAPE_Train          MAPE_Test")
print ("Linear Regression -         Train: ",Train_MAPE[0],"      Test: ",Test_MAPE[0])
print ("Random Forest Regression -  Train: ",Train_MAPE[1],"      Test: ",Test_MAPE[1])
print ("RF with Hyper parm tuning - Train: ",Train_MAPE[2],"      Test: ",Test_MAPE[2])
print ("XgBoost Regression -        Train: ",Train_MAPE[3],"      Test: ",Test_MAPE[3])

                                         MAPE_Train          MAPE_Test
Linear Regression -         Train:  0.05111821553507838       Test:  0.05129853195553662
Random Forest Regression -  Train:  0.006402364479940866       Test:  0.0094745062797144
RF with Hyper parm tuning - Train:  0.00256008913068328       Test:  0.006668597246162481
XgBoost Regression -        Train:  0.013531701373869272       Test:  0.015013366945789145


R2 Score

In [ ]:
from sklearn.metrics import r2_score

Train_r2=[]
Test_r2=[]

Train_r2.append(r2_score(y_train,lr_train_predictions))
Train_r2.append(r2_score(y_train,rndf_train_predictions))
Train_r2.append(r2_score(y_train,rndf_train_predictions_ht))
Train_r2.append(r2_score(y_train,xgb_train_predictions))

Test_r2.append(r2_score(y_test,lr_test_predictions))
Test_r2.append(r2_score(y_test,rndf_test_predictions))
Test_r2.append(r2_score(y_test,rndf_test_predictions_ht))
Test_r2.append(r2_score(y_test,xgb_test_predictions))

print ("  Model                                 R2_Train          R2_Test")
print ("Linear Regression -         Train: ",Train_r2[0],"      Test: ",Test_r2[0])
print ("Random Forest Regression -  Train: ",Train_r2[1],"      Test: ",Test_r2[1])
print ("RF with Hyper parm tuning - Train: ",Train_r2[2],"      Test: ",Test_r2[2])
print ("XgBoost Regression -        Train: ",Train_r2[3],"      Test: ",Test_r2[3])

  Model                                 R2_Train          R2_Test
Linear Regression -         Train:  0.5238275687338817       Test:  0.4554391960421752
Random Forest Regression -  Train:  0.9833867693044931       Test:  0.9658345119631042
RF with Hyper parm tuning - Train:  0.9975182331394794       Test:  0.981306376994859
XgBoost Regression -        Train:  0.9606075112616317       Test:  0.9482526178156911


Random forest with hyper paramter tuning and with out hyper parmeter tuning also giving almost same results.

# Conclusion :

By looking at above observations we can say XGboost regressor is less overfitting compare to remaning two models. But Random forest regressor giving the best results in terms of error.

In [ ]:
from tabulate import tabulate

l = [["Linear Regression",Train_RMSE[0],Test_RMSE[0],Train_MAPE[0],Test_MAPE[0],Train_r2[0],Test_r2[0]],
     ["Random Forest",    Train_RMSE[1],Test_RMSE[1],Train_MAPE[1],Test_MAPE[1],Train_r2[1],Test_r2[1]],
     ["RF with Par tuning",Train_RMSE[2],Test_RMSE[2],Train_MAPE[2],Test_MAPE[2],Train_r2[2],Test_r2[2]],
     ["XGBoost Regressor",Train_RMSE[3],Test_RMSE[3],Train_MAPE[3],Test_MAPE[3],Train_r2[3],Test_r2[3]]]
table = tabulate(l, headers=['Model', 'RMSE(Train)', 'RMSE(Test)','MAPE(Train)','MAPE(Test)', 'R2(Train)', 'R2(Test)'], tablefmt='orgtbl')

print(table)

| Model              |   RMSE(Train) |   RMSE(Test) |   MAPE(Train) |   MAPE(Test) |   R2(Train) |   R2(Test) |
|--------------------+---------------+--------------+---------------+--------------+-------------+------------|
| Linear Regression  |      2839.21  |     2977.11  |    0.0511182  |   0.0512985  |    0.523828 |   0.455439 |
| Random Forest      |       530.326 |      745.702 |    0.00640236 |   0.00947451 |    0.983387 |   0.965835 |
| RF with Par tuning |       204.973 |      551.593 |    0.00256009 |   0.0066686  |    0.997518 |   0.981306 |
| XGBoost Regressor  |       816.625 |      917.732 |    0.0135317  |   0.0150134  |    0.960608 |   0.948253 |


In [ ]:
y_pred_lr = np.append(y_pred_train_lr, y_pred_test_lr)
y__pred_Rf = np.append(y_pred_train_rf, y_pred_test_rf)
y_pred_Xgb = np.append(y_pred_train_xg, y_pred_test_xg)
final_op = np.hstack((X,y.reshape(-1,1), y_pred_lr.reshape(-1,1), y_pred_Rf.reshape(-1,1), y_pred_Xgb.reshape(-1,1)))
print(final_op.shape)
df = (pd.DataFrame(data=final_op,columns=["Date time","Volume (mV)","Light_Level (Ohms)","Temperature-DHT (Celsius)","Pressure (Hectopascal)","Temperature-BMP (Celsius)","Relative_Humidity (%)","Carbon_Monoxide (Ohms)","Nitrogen_Dioxide (Ohms)","Air_Quality (Ohms)","AirQuality_Predicted_LR (Ohms)","AirQuality_Predicted_RF (Ohms)","AirQuality_Predicted_XGB (Ohms"]))
df.head()
df.to_csv("AirPi_Output.csv")


(14572, 13)


,Date time,Volume (mV),Light_Level (Ohms),Temperature-DHT (Celsius),Pressure (Hectopascal),Temperature-BMP (Celsius),Relative_Humidity (%),Carbon_Monoxide (Ohms),Nitrogen_Dioxide (Ohms),Air_Quality (Ohms),AirQuality_Predicted_LR (Ohms),AirQuality_Predicted_RF (Ohms),AirQuality_Predicted_XGB (Ohms
0,2016-11-27 17:22:45,133.065,5064.14,24.05,1024.43,24.13,37.78,283808,14849.6,40776.5,37262.4,36751,37908.9
1,2016-11-27 17:27:45,96.9355,5075.23,24.525,1024.49,24.52,36.88,286562,15359.9,40661.1,40363.8,40835.1,41262
2,2016-11-27 17:32:46,97.9032,5074.1,25.045,1024.58,24.895,35.705,293986,15772.1,40666.6,40140.1,42581,42907.8
3,2016-11-27 17:37:46,105.645,5086.33,25.335,1024.63,25.08,34.81,298588,16150.7,40574.7,46297.1,45422,45316.4
4,2016-11-27 17:42:46,130.806,5086.34,26.75,1024.74,25.25,34.435,299601,16506.6,40546.1,40030.1,41007.7,40661.7
